In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)

In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff = 200
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts().loc[lambda x : x<cutoff].index

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df['CLASSIFICATION'].value_counts().loc[lambda x : x>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff = 100
classifications_to_replace = application_df['CLASSIFICATION'].value_counts().loc[lambda x : x<cutoff].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`

cat_list = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

application_df_converted = pd.get_dummies(application_df, prefix=cat_list)
application_df_converted.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [10]:
# Split our preprocessed data into our features and target arrays
y = application_df_converted['IS_SUCCESSFUL'].values
X = application_df_converted.drop(columns="IS_SUCCESSFUL").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
# Changing the model to have less epochs and a little more nodes and using a validation split from the first time

nn = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_dim=len(X_train_scaled[0])),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               6400      
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 16,769
Trainable params: 16,769
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50, validation_split=0.3)

Epoch 1/50
563/563 [==============================] - 2s 3ms/step - loss: 0.5661 - accuracy: 0.7269 - val_loss: 0.5649 - val_accuracy: 0.7197
Epoch 2/50
563/563 [==============================] - 2s 3ms/step - loss: 0.5482 - accuracy: 0.7363 - val_loss: 0.5650 - val_accuracy: 0.7208
Epoch 3/50
563/563 [==============================] - 2s 3ms/step - loss: 0.5469 - accuracy: 0.7360 - val_loss: 0.5632 - val_accuracy: 0.7192
Epoch 4/50
563/563 [==============================] - 2s 3ms/step - loss: 0.5446 - accuracy: 0.7370 - val_loss: 0.5585 - val_accuracy: 0.7232
Epoch 5/50
563/563 [==============================] - 2s 3ms/step - loss: 0.5429 - accuracy: 0.7375 - val_loss: 0.5584 - val_accuracy: 0.7232
Epoch 6/50
563/563 [==============================] - 2s 3ms/step - loss: 0.5422 - accuracy: 0.7366 - val_loss: 0.5599 - val_accuracy: 0.7248
Epoch 7/50
563/563 [==============================] - 1s 3ms/step - loss: 0.5405 - accuracy: 0.7403 - val_loss: 0.5583 - val_accuracy: 0.7240
Epoch 

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5674 - accuracy: 0.7248 - 322ms/epoch - 1ms/step
Loss: 0.5673655271530151, Accuracy: 0.724781334400177


In [16]:
# Export our model to HDF5 file
nn.save('Results/AlphabetSoupCharity_2.h5')

In [17]:
# Third try on the nn model changing the layers and the amount of epochs 
nn_3 = tf.keras.models.Sequential([
    tf.keras.layers.Dense(100, activation='relu', input_dim=len(X_train_scaled[0])),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
# Check the structure of the model
nn_3.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 100)               5000      
                                                                 
 dense_5 (Dense)             (None, 50)                5050      
                                                                 
 dense_6 (Dense)             (None, 1)                 51        
                                                                 
Total params: 10,101
Trainable params: 10,101
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Compile the model
nn_3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [20]:
# Train the model
fit_model = nn_3.fit(X_train_scaled, y_train, epochs=200, validation_split=0.3)

Epoch 1/200
563/563 [==============================] - 3s 5ms/step - loss: 0.5683 - accuracy: 0.7255 - val_loss: 0.5690 - val_accuracy: 0.7221
Epoch 2/200
563/563 [==============================] - 2s 3ms/step - loss: 0.5490 - accuracy: 0.7339 - val_loss: 0.5604 - val_accuracy: 0.7217
Epoch 3/200
563/563 [==============================] - 2s 3ms/step - loss: 0.5464 - accuracy: 0.7346 - val_loss: 0.5668 - val_accuracy: 0.7190
Epoch 4/200
563/563 [==============================] - 1s 3ms/step - loss: 0.5443 - accuracy: 0.7361 - val_loss: 0.5577 - val_accuracy: 0.7223
Epoch 5/200
563/563 [==============================] - 2s 3ms/step - loss: 0.5423 - accuracy: 0.7369 - val_loss: 0.5640 - val_accuracy: 0.7183
Epoch 6/200
563/563 [==============================] - 2s 3ms/step - loss: 0.5423 - accuracy: 0.7366 - val_loss: 0.5587 - val_accuracy: 0.7264
Epoch 7/200
563/563 [==============================] - 1s 3ms/step - loss: 0.5412 - accuracy: 0.7389 - val_loss: 0.5612 - val_accuracy: 0.7188

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.7310 - accuracy: 0.7237 - 327ms/epoch - 1ms/step
Loss: 0.7309625148773193, Accuracy: 0.7237317562103271


In [22]:
# Export our model to HDF5 file
nn.save('Results/AlphabetSoupCharity_3.h5')